<a href="https://colab.research.google.com/github/Tekleab15/Regularized_Auto_Encoder/blob/main/RAE_implementation_using_MNIST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  **Regularized Autoencoder - RAE Using MNIST dataset **

*Implementation of RAE(Regularized Autoencoders) as per the specification on the paper THE NEURAL CODING FRAMEWORK FOR LEARNING GENERATIVE MODELS  *

In [20]:
import tensorflow as tf
import numpy as np
import matplotlib as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, InputLayer
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import Callback
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KNeighborsClassifier

In [17]:
def preprocess_dataset(x_train, x_test):
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    x_train = (x_train > 0.5).astype('float32')
    x_test = (x_test > 0.5).astype('float32')
    x_train = x_train.reshape(-1, 784)
    x_test = x_test.reshape(-1, 784)
    return x_train, x_test

In [7]:
class GradientClippingCallback(Callback):
    def __init__(self, clipnorm):
        super(GradientClippingCallback, self).__init__()
        self.clipnorm = clipnorm

    def on_batch_end(self, batch, logs=None):
        for layer in self.model.layers:
            if hasattr(layer, 'kernel'):
                weights, bias = layer.get_weights()
                weights_norm = tf.norm(weights, ord='euclidean')
                if weights_norm > self.clipnorm:
                    weights = weights * self.clipnorm / weights_norm
                layer.set_weights([weights, bias])

In [8]:
# Load and preprocess the MNIST dataset, including labels
(x_train_raw, y_train), (x_test_raw, y_test) = mnist.load_data()
x_train, x_test = preprocess_dataset(x_train_raw, x_test_raw)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
input_shape = (784,)
latent_dim = 20
hidden_layer_size = 360

encoder = Sequential(name="encoder")
encoder.add(InputLayer(shape=input_shape))
encoder.add(Dense(hidden_layer_size, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.05),kernel_regularizer=regularizers.l2(1e-4)))
encoder.add(Dense(latent_dim, activation='sigmoid'))

decoder = Sequential(name="decoder")
decoder.add(InputLayer(shape=(latent_dim,)))
decoder.add(Dense(hidden_layer_size, activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
decoder.add(Dense(input_shape[0], activation='sigmoid'))
decoder.add(Reshape(input_shape))

rae = Sequential([encoder, decoder], name="RAE")

In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
rae.compile(optimizer=optimizer, loss='binary_crossentropy')
print(rae.summary())

Model: "RAE"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ encoder (Sequential)                 │ (None, 20)                  │         289,820 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder (Sequential)                 │ (None, 784)                 │         290,584 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 580,404 (2.21 MB)

 Trainable params: 580,404 (2.21 MB)

 Non-trainable params: 0 (0.00 B)

None


In [14]:
gradient_clipping_callback = GradientClippingCallback(clipnorm=5.0)
rae.fit(x_train, x_train, epochs=50, batch_size=200, validation_data=(x_test, x_test), callbacks=[gradient_clipping_callback])
# rae.fit(x_train, x_train, epochs=50, batch_size=256, validation_data=(x_test, x_test))

Epoch 1/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step - loss: 0.3535 - val_loss: 0.2251
Epoch 2/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.2067 - val_loss: 0.1790
Epoch 3/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 0.1750 - val_loss: 0.1614
Epoch 4/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - loss: 0.1599 - val_loss: 0.1526
Epoch 5/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - loss: 0.1526 - val_loss: 0.1464
Epoch 6/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 0.1464 - val_loss: 0.1409
Epoch 7/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - loss: 0.1408 - val_loss: 0.1357
Epoch 8/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - loss: 0.1363 - val_loss: 0.1320
Epoch 9/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - loss: 0.1325 - val_loss: 0.1279
Epoch 10/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 0.1285 - val_loss: 0.1240
Epoch 11/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - loss: 0.1248 - val_loss: 0.1216
Epoch 12/50
300/300 ━━━━━━━━━━━━━━━

In [18]:
# Implementing and fitting the GMM model
z_train = encoder.predict(x_train)
gmm = GaussianMixture(n_components=75, covariance_type='full').fit(z_train)

sampled_latent = gmm.sample(n_samples=100)[0]
generated_samples = decoder.predict(sampled_latent)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [24]:
log_likelihood = gmm.score_samples(z_train)
average_log_likelihood = np.mean(log_likelihood)
print("Average Log Likelihood: ", average_log_likelihood)

Average Log Likelihood:  10.37270784035212


In [29]:
# Encode test data
z_test = encoder.predict(x_test)

# Calculate the log likelihood of the latent representations
log_likelihood_test = gmm.score_samples(z_test)
average_log_likelihood_test = np.mean(log_likelihood_test)
print("Average Log Likelihood for Test Data: ", average_log_likelihood_test)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Average Log Likelihood for Test Data:  9.886330946091821


In [34]:
def evaluate_model_per_sample(model, x_test, y_test, gmm):
    reconstructions = model.predict(x_test)
    bce_mean = tf.keras.losses.BinaryCrossentropy()(x_test, reconstructions).numpy()
    mse_mean = tf.keras.losses.MeanSquaredError()(x_test, reconstructions).numpy()
    bce_per_sample = bce_mean * x_test.shape[1]
    mse_per_sample = mse_mean * x_test.shape[1]
    log_likelihood_per_sample = - bce_per_sample

    z_test = encoder.predict(x_test)
    z_train = encoder.predict(x_train)
    z_train = z_train / np.linalg.norm(z_train, axis=1, keepdims=True)
    z_test = z_test / np.linalg.norm(z_test, axis=1, keepdims=True)

    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(z_train, y_train)
    y_pred = knn.predict(z_test)
    classification_error = 100 * (1 - np.mean(y_pred == y_test))

    # Sample new data points from the GMM for evaluation
    sampled_latent = gmm.sample(n_samples=100)[0]
    generated_samples = decoder.predict(sampled_latent)

    log_likelihood_train = gmm.score_samples(z_train)
    mean_log_likelihood_train = np.mean(log_likelihood_train)

    log_likelihood_test = gmm.score_samples(z_test)
    mean_log_likelihood_test = np.mean(log_likelihood_test)

    print("GMM Test likelyhood: ",round(mean_log_likelihood_test, 2), "GMM Train likelyhood: ",round(mean_log_likelihood_train,2))

    return {
        "MSE per Sample": round(mse_per_sample, 2),
        "BCE per Sample": round(bce_per_sample, 2),
        "Log-Likelihood per Sample": round(log_likelihood_per_sample, 2),
        "Classification Error (%)": round(classification_error, 2),
        # "Generated Samples": generated_samples
    }

evaluation_results = evaluate_model_per_sample(rae, x_test, y_test, gmm)
print(evaluation_results)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
GMM Test likelyhood:  -9.61 GMM Train likelyhood:  -9.64
{'MSE per Sample': 18.66, 'BCE per Sample': 62.38, 'Log-Likelihood per Sample': -62.38, 'Classification Error (%)': 3.84}
